# Обучение модели на отобранных признаках

## 1 Импорт библиотек, инициализация глобальных констант

### 1.1 Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
from colorama import Fore, Back, Style
import joblib

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_percentage_error

import lightgbm as lgb
from lightgbm import log_evaluation
import shap

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'lightgbm'

### 1.2 Инициализация констант

In [ ]:
# целевая переменная
TARGET_NAME = 'Цена_квм'
# размер отложенной выборки
TEST_SIZE = 0.15
# random state для воспроизводимости
RANDOM_STATE = 42

## 2 Подготовка данных

In [ ]:
df = pd.read_csv('houseprices_final.csv')
metros = pd.read_csv('metro_coords.csv')
df.head()

### 2.1 Кодирование станций метро

In [ ]:
metro_stations = np.append(metros['Станция'].values, ['нет'])

le = LabelEncoder()

le.fit(metro_stations.reshape(-1, 1))
for i in range(1, 4):
    df[f'Метро_{i}'] = le.transform(df[f'Метро_{i}'])
# сохраняем
le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
with open("metro_stations.pkl", "wb") as f:
    pickle.dump(le_dict, f)

### 2.2 Кодирование категориальных переменных

In [ ]:
cat_features = ['Район', 'Количество_комнат', 'Ремонт', 'Тип_дома']

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df[cat_features] = encoder.fit_transform(df[cat_features])
# сохраняем
with open("categorical_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

### 2.3 Отложенная выборка

In [ ]:
train, test = train_test_split(df,
                               test_size=TEST_SIZE,
                               stratify=df['Количество_комнат'],
                               random_state=RANDOM_STATE)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

## 3 Обучение модели

### 3.1 Функция для обучения и получения предсказаний

In [ ]:
def lgb_training(train_df, params, features, test_df=None, verbose=250):
    """
    Функция для обучения модели.
    
    Параметры
    ----------
    train_df - обучающая выборка
    params - набор гиперпараметров
    features - признаки
    test_df - тестовая выборка
    verbose - вывод результатов
    
    Возвращает
    -------
    score_list - MAPE на кросс-валидации для каждого фолда
    test_preds - предсказания на тестовой выборке, если тестовая выборка задана
    oof_preds - предсказания для обучающей выборки
    models - список обученных моделей
    """
    test_preds = []
    if test_df is not None:
        test_preds = np.zeros(len(test_df))
        test_df = test_df[features]
    target = train_df.loc[:, TARGET_NAME].values

    categorical_feature = list(set(features) & set(cat_features))
    models = []
    score_list = []
    oof_preds = np.zeros(len(target))

    kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
        X_train = train_df.loc[train_index, features]
        X_val = train_df.loc[val_index, features]
        y_train = target[train_index]
        y_val = target[val_index]

        lgb_train = lgb.Dataset(X_train,
                                y_train,
                                categorical_feature=categorical_feature)
        lgb_valid = lgb.Dataset(X_val,
                                y_val,
                                categorical_feature=categorical_feature)

        callbacks = [lgb.log_evaluation(period=verbose)]
        model = lgb.train(params=params,
                          train_set=lgb_train,
                          num_boost_round=15000,
                          valid_sets=[lgb_train, lgb_valid],
                          categorical_feature=categorical_feature,
                          callbacks=callbacks)

        preds = model.predict(X_val)
        oof_preds[val_index] = preds
        score = mean_absolute_percentage_error(y_val, preds)

        if test_df is not None:
            test_preds += model.predict(test_df) / kf.n_splits

        if verbose:
            print(
                f"{Fore.BLACK}{Style.BRIGHT}FOLD: {fold}, MAPE: {score:.5f} {Style.RESET_ALL}"
            )
        score_list.append(score)

        models.append(model)

    return score_list, test_preds, oof_preds, models

### 3.2 Гиперпараметры
По сравнению с бейзлайном немного уменьшил learning_rate

In [ ]:
params = {
    'objective': 'mape',
    'learning_rate': 0.25,
    'reg_lambda': 5,
    'random_state': 42,
    'early_stopping_rounds': 300,
    'feature_fraction': 0.4,
    'verbose': -1
}

### 3.3 Отобранные признаки
Признаки адрес, этаж, количество комнат, общая площадь, этаж, количество этажей в доме есть в каждом объявлении, поэтому решено было получать их от пользователя.

На основе важности функций на основе перестановки определил наиболее важные созданные признаки

In [ ]:
features = [
    'Район', 'Количество_комнат', 'Общая_площадь', 'Ремонт', 'Этаж',
    'Этажей_в_доме', 'Тип_дома', 'Расстояние_до_Краснопресненская',
    'Координаты_кластер', 'Метро_1', 'Метро_2', 'Метро_3',
    'Общая_площадь_минус_мин', 'Расстояние_до_Кремля', 'Расстояние_до_Киевская'
]

### 3.4 Обучение, сохранение моделей

In [ ]:
score_list, test_preds, oof_preds, models = lgb_training(train,
                                                         params,
                                                         features,
                                                         test,
                                                         verbose=500)
test_score = mean_absolute_percentage_error(test[TARGET_NAME], test_preds)

print(f"{Fore.GREEN}{Style.BRIGHT}\nMAPE на кросс-валидации:", end=' ')
print(f"{np.mean(score_list):.5f} +- {np.std(score_list):.5f}")

print(f"{Fore.RED}MAPE на тестовой выборке: {test_score:.5f}{Style.RESET_ALL}")

In [ ]:
sns.histplot(np.abs(test[TARGET_NAME] - test_preds) / test[TARGET_NAME] * 100)
plt.title("MAPE для тестовых данных")
plt.xlabel("MAPE, %")
plt.ylabel("Количество")
sns.despine()
plt.show()

### 3.5 Сохранение
Возможно 2 подхода:
*  использовать 5 моделей, каждая из которых обучалась на 80% тренировочных данных
*  обучить 1 модель на всей тренировочной выборке

Посмотрим ошибку на тренировочной выборке при 2 способе. Для этого обучим зададим количество итераций, равное медиане лучших итерации на каждом фолде


In [ ]:
iterations = [model.best_iteration for model in models]
n_iter = int(np.median(iterations)) + 1
print('Лучшая итерация для каждого фолда:')
print(iterations)

In [ ]:
params['metric'] = None
params['early_stopping_rounds'] = None

model = lgb.train(params=params,
                  train_set=lgb.Dataset(train[features], train[TARGET_NAME]),
                  num_boost_round=n_iter,
                  categorical_feature=cat_features,
                  callbacks=[lgb.log_evaluation(period=0)])

test_preds=model.predict(test[features])
test_score=mean_absolute_percentage_error(test[TARGET_NAME],test_preds)
print(f"{Fore.RED}{Style.BRIGHT}MAPE на тестовой выборке: {test_score:.5f}{Style.RESET_ALL}")

MAPE при таком способе оказалась выше на 0.002 по сравнению с первым способом. Думаю, это связано с тем, что предсказания моделей, обученных на 80% тренировочных данных (4 фолдах) хоть сколько-то некоррелированы, благодаря чему при усреднении удается снизить ошибку за счет уменьшения разброса

In [ ]:
# сохранение
for fold in range(5):
    joblib.dump(models[fold], f'lgb_model_{fold}')

### 3.6 Эксперименты над улучшением модели


Для каждого примера из тренировочной выборки посчитаем значение ошибки на кросс-валидации

In [ ]:
train['MAPE'] = np.abs(train[TARGET_NAME] - oof_preds) / train[TARGET_NAME]

train.sort_values(by='MAPE', ascending=False).head(10)

Попробуем заново обучить модель, предварительно удалив из обучающей выборки объявления, ошибка на которых больше определенного порога

In [ ]:
THRESHOLDS = [0.8, 0.9, 1, 1.2, 1.5]
for thr in THRESHOLDS:
    tmp_train = train.loc[train['MAPE'] < thr].reset_index(drop=True)
    score_list, test_preds, _, _ = lgb_training(tmp_train,
                                                params,
                                                features,
                                                test,
                                                verbose=0)
    test_score = mean_absolute_percentage_error(test[TARGET_NAME], test_preds)

    print(f"{Fore.BLACK}{Style.BRIGHT}Порог для удаления: {thr*100:.0f}%")
    print(f"{Fore.GREEN}{Style.BRIGHT}MAPE на кросс-валидации:", end=' ')
    print(f"{np.mean(score_list):.5f} +- {np.std(score_list):.5f}")

    print(
        f"{Fore.RED}MAPE на тестовой выборке: {test_score:.5f}{Style.RESET_ALL}"
    )
    print()

## 4 Важность признаков и интерпретация предсказаний

In [ ]:
explainer = shap.TreeExplainer(models[4])
shap_values = explainer(test[features])

In [ ]:
shap.plots.bar(shap_values, max_display=None)

Самым важным, как и при permutation importance, оказался Район. Как было выяснено в ходе разведывательного анализа данных, количество комнат слабо влияет на стоимость квадратного метра, что и видно на графике

In [ ]:
shap.plots.beeswarm(shap_values, max_display=None)

Можно сделать следующие выводы:
*  чем дальше от Кремля находится квартира, тем дешевле стоимость квадратного метра
*  чем дальше от метро Киевская находится квартира, тем дешевле стоимость квадратного метра. Возможно, здесь важно учесть, что Киевская ближе всего к Москва-Сити (это подтверждает тот факт, что для модели так же важно и расстояние до метро Краснопресненская) или к Арбату
*  чем больше этажей в доме, тем дороже стоимость квадратного метра. Я посмотрел [список самих высоких зданий Москвы](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%81%D0%B0%D0%BC%D1%8B%D1%85_%D0%B2%D1%8B%D1%81%D0%BE%D0%BA%D0%B8%D1%85_%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B9_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B) и почти во всех из них продается элитное жилье
*  чем ниже этаж на котором расположена квартира, тем дешевле стоимость квадратного метра. [Возможно](https://realty.rbc.ru/news/577d242c9a7947a78ce91b8b) это связано с тем, что традиционно стоимость квадратного метра на первом этаже всегда ниже, чем в среднем по дому.